In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from mlp import get_batch
from torch.nn import ReLU, Sigmoid
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# def truncate(pscores, thresh):
#     return np.clip(pscores, thresh, 1 - thresh)

# def ipw_ht(Z, Y, pscores):

#     weights = Z / pscores + (1 - Z) / (1 - pscores)

#     output = np.sum(weights * Y) / np.sum(weights)
#     return output


# def ipw_hayek(Z, Y, pscores):

#     t1 = Z / pscores
#     t2 = (1 - Z) / (1 - pscores)

#     return np.sum(t1 * Y) / np.sum(t1) - np.sum(t2 * Y) / np.sum(t2)

# hyperparameters = {
#     "is_causal": True,
#     "num_causes": 8,
#     "prior_mlp_hidden_dim": 8,
#     "num_layers": 3,
#     "noise_std": 0.1,
#     "y_is_effect": True,
#     "pre_sample_weights": False,
#     "prior_mlp_dropout_prob": 0.5,
#     "pre_sample_causes": False,
#     "prior_mlp_activations": Sigmoid,
#     "block_wise_dropout": True,
#     "init_std": 1.0,
#     'prior_mlp_scale_weights_sqrt': True,
#     'sampling': 'normal',
#     'in_clique': False,
#     'sort_features': False,
#     'random_feature_rotation': False,
#     'new_mlp_per_example': True
# } 

# x, y, _ = get_batch(batch_size=100, seq_len=10000, num_features=8, hyperparameters=hyperparameters,
#           num_outputs=3, sampling="mixed")
# x = x.numpy()
# y = y.numpy()
# print(x.shape, y.shape)

# plt.scatter(x[:,0,0], x[:,0,1], c=y[:,0,0], s=2)

In [ ]:
# ATEs = []
# ATE_hayeks = []
# for i in range(100):
#     X = x[:,i,:]
#     Y0 = y[:,i,0]
#     Z = y[:,i,1]
#     Z = Z > np.median(Z)
#     Y1 = Y0 + 1#np.abs(y[:,i,2]) # 1
#     Y = np.where(Z, Y1, Y0)
#     ATE = np.mean(Y1) - np.mean(Y0)
#     # print(ATE)
#     ATEs.append(ATE)

#     pscore_model = LogisticRegression()
#     pscore_model.fit(X, Z)
#     pscores = pscore_model.predict_proba(X)[:, 1]
#     pscores = truncate(pscores, 0.01)

#     ATE_hayek = ipw_hayek(Z, Y, pscores)
#     ATE_hayeks.append(ATE_hayek)

In [ ]:
hyperparameters = {
    "is_causal": True,
    "num_causes": 1000,
    "prior_mlp_hidden_dim": 1000,
    "num_layers": 2,
    "noise_std": 0.1,
    "y_is_effect": True,
    "pre_sample_weights": False,
    "prior_mlp_dropout_prob": 0,
    "pre_sample_causes": False,
    "prior_mlp_activations": ReLU,
    "block_wise_dropout": False,
    "init_std": 1.0,
    'prior_mlp_scale_weights_sqrt': True,
    'sampling': 'normal',
    'in_clique': False,
    'sort_features': False,
    'random_feature_rotation': False,
    'new_mlp_per_example': True
}

_, x, _ = get_batch(batch_size=100, seq_len=10000, num_features=8, hyperparameters=hyperparameters,
          num_outputs=2, sampling="mixed")
# _, u, _ = get_batch(batch_size=100, seq_len=10000, num_features=8, hyperparameters=hyperparameters,
#           num_outputs=3, sampling="mixed")

# x = x.numpy()
# u = u.numpy()
x = x.detach()
# u = u.detach()

# print(x.shape, y.shape)

# plt.scatter(x[:,0,0], x[:,0,1], c=y[:,0,0], s=2)

In [ ]:
x.shape#, u.shape

In [ ]:
# u.mean()

In [ ]:
x.mean()

In [ ]:
causes = [[x[:, i, np.newaxis, :]] for i in range(100)]

In [ ]:
# make list of 10000 x, u pairs
causes = [[x[:,i,np.newaxis,:], u[:,i,np.newaxis,:]] for i in range(100)]

In [ ]:
hyperparameters['num_causes'] = x.shape[-1]
_, y, _ = get_batch(batch_size=100, seq_len=10000, num_features=8, hyperparameters=hyperparameters,
          num_outputs=2, sampling="mixed", causes=causes)

In [ ]:
plt.hist(y[:,1,0].numpy(), bins=50)
plt.show()

In [ ]:
for i in range(10):
    fig, axs = plt.subplots(1,3,figsize=(15, 5))
    axs[0].scatter(x[:,i,0].numpy(), y[:,i,0].numpy(), s=2)
    axs[1].scatter(x[:,i,1].numpy(), y[:,i,0].numpy(), s=2)
    axs[2].scatter(x[:,i,0].numpy(), x[:,i,1].numpy(), c=y[:,i,0].numpy(), s=2)

    plt.show()

In [ ]:
y.shape

In [ ]:
x.shape

In [ ]:
grid = [2,4,6,8,10,20,30,40,50,60,70,80,90,100]
avg_skew_x1 = []
avg_skew_x2 = []
avg_skew_y = []
avg_kurtosis_x1 = []
avg_kurtosis_x2 = []
avg_kurtosis_y = []
r2_results = {}
for i in grid:
    hyperparameters = {
        "is_causal": True,
        "num_causes": i,
        "prior_mlp_hidden_dim": i,
        "num_layers": 2,
        "noise_std": 0.1,
        "y_is_effect": True,
        "pre_sample_weights": False,
        "prior_mlp_dropout_prob": 0.5,
        "pre_sample_causes": False,
        "prior_mlp_activations": ReLU,
        "block_wise_dropout": False,
        "init_std": 1.0,
        'prior_mlp_scale_weights_sqrt': True,
        'sampling': 'normal',
        'in_clique': False,
        'sort_features': False,
        'random_feature_rotation': False,
        'new_mlp_per_example': True
    }

    _, x, _ = get_batch(batch_size=500, seq_len=5000, num_features=8, hyperparameters=hyperparameters,
            num_outputs=2, sampling="mixed")
    # _, u, _ = get_batch(batch_size=100, seq_len=10000, num_features=8, hyperparameters=hyperparameters,
    #         num_outputs=3, sampling="mixed")

    # x = x.numpy()
    # u = u.numpy()
    x = x.detach()
    # u = u.detach()
    
    r2_results[i] = np.zeros((x.shape[1], 10))  # Store R^2 for each degree

    # make list of 10000 x, u pairs
    causes = [[x[:,i,np.newaxis,:]] for i in range(x.shape[1])]

    hyperparameters["num_causes"] = x.shape[-1]
    _, y, _ = get_batch(batch_size=500, seq_len=5000, num_features=8, hyperparameters=hyperparameters,
            num_outputs=1, sampling="mixed", causes=causes)
    x = x.numpy()
    skew_arr_x1 = []
    skew_arr_x2 = []
    skew_arr_y = []
    kurtosis_arr_x1 = []
    kurtosis_arr_x2 = []
    kurtosis_arr_y = []
    for dataset in range(x.shape[1]):
        x1 = x[:, dataset, 0]
        x2 = x[:, dataset, 1]
        y0 = y[:, dataset].numpy()
        # get skew of x1, x2, y
        skew_x1 = np.abs(np.mean((x1 - np.mean(x1))**3) / (np.std(x1)**3))
        skew_x2 = np.abs(np.mean((x2 - np.mean(x2))**3) / (np.std(x2)**3))
        skew_y = np.abs(np.mean((y0 - np.mean(y0))**3) / (np.std(y0)**3))
        skew_arr_x1.append(skew_x1)
        skew_arr_x2.append(skew_x2)
        skew_arr_y.append(skew_y)
        # get kurtosis of x1, x2, y
        kurtosis_x1 = np.mean((x1 - np.mean(x1))**4) / (np.std(x1)**4) - 3
        kurtosis_x2 = np.mean((x2 - np.mean(x2))**4) / (np.std(x2)**4) - 3
        kurtosis_y = np.mean((y0 - np.mean(y0))**4) / (np.std(y0)**4) - 3
        kurtosis_arr_x1.append(kurtosis_x1)
        kurtosis_arr_x2.append(kurtosis_x2)
        kurtosis_arr_y.append(kurtosis_y)
        
        X_train = x[:, dataset, :]
        y_train = y[:, dataset].numpy()
        # fit polynomial regression model for different degrees
        
        for degree in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
            from sklearn.preprocessing import PolynomialFeatures
            from sklearn.linear_model import LinearRegression
            
            poly = PolynomialFeatures(degree=degree)
            X_poly_train = poly.fit_transform(X_train)
            
            model = LinearRegression()
            model = model.fit(X_poly_train, y_train)
            
            # You can calculate metrics like MSE or R^2 here if needed
            r2_results[i][dataset, degree-1] = model.score(X_poly_train, y_train)
        
    skew_arr_x1 = np.array(skew_arr_x1)
    skew_arr_x2 = np.array(skew_arr_x2)
    skew_arr_y = np.array(skew_arr_y)
    kurtosis_arr_x1 = np.array(kurtosis_arr_x1)
    kurtosis_arr_x2 = np.array(kurtosis_arr_x2)
    kurtosis_arr_y = np.array(kurtosis_arr_y)
    avg_skew_x1.append(np.median(skew_arr_x1))
    avg_skew_x2.append(np.median(skew_arr_x2))
    avg_skew_y.append(np.median(skew_arr_y))
    avg_kurtosis_x1.append(np.median(kurtosis_arr_x1))
    avg_kurtosis_x2.append(np.median(kurtosis_arr_x2))
    avg_kurtosis_y.append(np.median(kurtosis_arr_y))
    
    
    
    # print("Skewness for Width:", i)
    # plt.hist(skew_arr_x1, bins=20, alpha=0.5, label='X1 Skewness', histtype='step', linewidth=4)
    # plt.hist(skew_arr_x2, bins=20, alpha=0.5, label='X2 Skewness', histtype='step', linewidth=4)
    # plt.hist(skew_arr_y, bins=20, alpha=0.5, label='Y Skewness', histtype='step', linewidth=4)
    # plt.legend()
    # plt.title(f'Skewness Distribution for Width {i}')
    # plt.show()
    # print("Kurtosis for Width:", i)
    # plt.hist(kurtosis_arr_x1, bins=20, alpha=0.5, label='X1 Kurtosis', histtype='step', linewidth=4)
    # plt.hist(kurtosis_arr_x2, bins=20, alpha=0.5, label='X2 Kurtosis', histtype='step', linewidth=4)
    # plt.hist(kurtosis_arr_y, bins=20, alpha=0.5, label='Y Kurtosis', histtype='step', linewidth=4)
    # plt.legend()
    # plt.title(f'Kurtosis Distribution for Width {i}')
    # plt.show()
    
# lineplot of average skewness vs width
plt.figure(figsize=(10, 5))
plt.plot(grid, avg_skew_x1, marker='o', label='Average Skewness X1')
plt.plot(grid, avg_skew_x2, marker='o', label='Average Skewness X2')
plt.plot(grid, avg_skew_y, marker='o', label='Average Skewness Y')
plt.xlabel('Width of MLP')
plt.ylabel('Average Skewness')
plt.title('Average Skewness vs Width of MLP')
plt.legend()
plt.show()
# lineplot of average kurtosis vs width
plt.figure(figsize=(10, 5))
plt.plot(grid, avg_kurtosis_x1, marker='o', label='Average Kurtosis X1')
plt.plot(grid, avg_kurtosis_x2, marker='o', label='Average Kurtosis X2')
plt.plot(grid, avg_kurtosis_y, marker='o', label='Average Kurtosis Y')
plt.xlabel('Width of MLP')
plt.ylabel('Average Kurtosis')
plt.title('Average Kurtosis vs Width of MLP')
plt.legend()
plt.show()


In [ ]:
X_poly_train.shape

In [ ]:
r2_results[2][1,:]

In [ ]:
r2_results[2].shape

In [ ]:
r2_results[2]

In [ ]:
# lineplot of r2 results vs width
r2_results_avg = {degree: [] for degree in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
for degree in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    for width in grid:
        r2_results_avg[degree].append(np.mean(r2_results[width][:, degree-1]))
plt.figure(figsize=(10, 5))
for degree in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    plt.plot(grid, r2_results_avg[degree], marker='o', label=f'Average R^2 Degree {degree}')
plt.xlabel('Width of MLP')
plt.ylabel('Average R^2')
plt.title('Average R^2 vs Width of MLP')
plt.legend()
plt.show()

In [ ]:
# plot the r2 results for each width and degree with x axis degree and y axis r2
plt.figure(figsize=(10, 5))
for width in grid:
    plt.plot(range(1, 11), r2_results[width].mean(axis=0), marker='o', label=f'Width {width}')
plt.xlabel('Degree of Polynomial')
plt.ylabel('Average R^2')
plt.title('Average R^2 vs Degree of Polynomial for Different Widths of MLP')
plt.legend()
plt.show()

In [ ]:
# regress y on x
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

r2_arr = []
for dataset in range(100):
    regression_model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(x[:, dataset, :], y[:, dataset, 0], test_size=0.3, random_state=42)
    regression_model.fit(X_train, y_train)
    y_pred = regression_model.predict(X_test)
    r2_arr.append(regression_model.score(X_test, y_test))
r2_arr = np.array(r2_arr)

# plot distribution of R^2 values
plt.hist(r2_arr, bins=20)
plt.xlabel('R^2 Value')
plt.ylabel('Frequency')
plt.title('Distribution of R^2 Values from Linear Regression')
plt.show()

In [ ]:
rands = np.random.normal(0, 1, size=(10000, 8))

In [ ]:
y.shape

In [ ]:

np.corrcoef(y[:,0,0], rands[:,3])

In [ ]:
np.corrcoef(y[:,0,0], x[:,0,0])

In [ ]:
np.corrcoef(y[:,0,0], u[:,0,2])

In [ ]:
plt.scatter(y[:,0,0], x[:,0,0], s=2)

In [ ]:
plt.scatter(y[:,0,0], u[:,0,2], s=2)